In [13]:

import cv2
import numpy as np
import os
from matplotlib import pylab
import pylab as pl
from sklearn.metrics import confusion_matrix, accuracy_score 
# from sklearn.externals import joblib
import joblib

from scipy.cluster.vq import vq    

In [14]:
import os
os.getcwd()

'/Users/mohitbohra/Documents/Rohit/SOP'

In [15]:
clf, classes_names, stdSlr, k, voc = joblib.load("bovw21.pkl")

In [17]:
# TODO : Descriptor Types
brisk = cv2.BRISK_create(30)
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(nfeatures=30,scoreType=cv2.ORB_FAST_SCORE)
sift = cv2.xfeatures2d.SIFT_create()

In [18]:
test_path = 'Images/Test'

# window_size = [(32,32),(64,64),(128,128)]
window_size = [(128,128)]

images = [os.path.join(test_path,f) for f in os.listdir(test_path)]

dims = (512,512)

num_kpts=0

for w in window_size:
    print("Window : ",w)
    for image in images:
        print("Reading img : ",image)
        img = cv2.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(gray, (512, 512))
        
#         cv2.imshow("Image", gray)
#         cv2.waitKey(0)   
#         cv2.destroyAllWindows()
        
        height, width= resized_image.shape
#         print("Dimensions : ",width,height)
        window_number = 1
        windows = []

        y=0

        des_list = []

        while y<height:
            if(y+w[1]>dims[1]):
                break  
            x=0
            while x<width:
                if (x+w[0])>dims[0]:
                    break
                window_number+=1
                imgWindow = resized_image[y:y+w[1], x:x+w[0]]

                kpts, des = brisk.detectAndCompute(imgWindow, None)

                if len(kpts)==0:
                    x = x + w[0]
                    continue;

                des_list.append((window_number, des))
                num_kpts += 1
                # print(window_number," : ",x,",",y)
                # print(len(kpts),",",imgWindow.shape)
                
                # cv2.imshow("Window Image", imgWindow)
                # cv2.waitKey(0)   
                # cv2.destroyAllWindows()
                
                x = x + w[0]

            y = y+w[1]

        # For One Image, All Windows Scanned!

        # Stack all the descriptors vertically in a numpy array

        # TODO : Apply this Image to Train with BOVW generated histograms!

        # print("des_list : ",len(des_list[0]))

        descriptors = des_list[0][1]
        # print(descriptors)
        
        for window_number, descriptor in des_list[0:]:
            # print("check : ",descriptors.shape)
            descriptors = np.vstack((descriptors, descriptor))

        test_features = np.zeros((window_number, k), "float32")

        for i in range(num_kpts):
            words, distance = vq(des_list[i][1],voc)
            for w in words:
                test_features[i][w] += 1
                

        # Perform Tf-Idf vectorization
        nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*num_kpts+1) / (1.0*nbr_occurences + 1)), 'float32')

        test_features = stdSlr.transform(test_features)

        predictions =  [classes_names[i] for i in clf.predict(test_features)]

        print("Image : ",image)
        print ("prediction ="  + str(predictions))

Window :  (128, 128)
Reading img :  Images/Test/Img9.png
Image :  Images/Test/Img9.png
prediction =['Side', 'Side', 'Side', 'Side', 'Front', 'Front', 'Side', 'Side', 'Front', 'Front', 'Side', 'Side', 'Front', 'Front', 'Side', 'Side', 'Side']
Reading img :  Images/Test/Img8.png


IndexError: invalid index to scalar variable.